# Corretor Ortográfico em Python: aplicando técnicas de NLP

## 01. Explorando um projeto de NLP

### 04. Importando um corpus textual

In [1]:
with open("dados/artigos.txt") as f:
    artigos = f.read()

artigos[:500]

'\n\n\nimagem \n\nTemos a seguinte classe que representa um usuário no nosso sistema:\n\njava\n\nPara salvar um novo usuário, várias validações são feitas, como por exemplo: Ver se o nome só contém letras, [**o CPF só números**] e ver se o usuário possui no mínimo 18 anos. Veja o método que faz essa validação:\n\njava \n\nSuponha agora que eu tenha outra classe, a classe `Produto`, que contém um atributo nome e eu quero fazer a mesma validação que fiz para o nome do usuário: Ver se só contém letras. E aí? Vou'

### 05. Tokenização

In [ ]:
texto_exemplo = "Olá, tudo bem?"
tokens = texto_exemplo.split(" ")
print(len(tokens))
print(tokens)

3
['Olá,', 'tudo', 'bem?']


## 02. Utilizando NLTK para tokenizar um texto

### 02. Refinando a tokenização

[Natural Language Toolkit - NLTK](https://www.nltk.org/)

In [7]:
import nltk

lista_tokens = nltk.tokenize.word_tokenize(artigos)
lista_tokens[:50]

['imagem',
 'Temos',
 'a',
 'seguinte',
 'classe',
 'que',
 'representa',
 'um',
 'usuário',
 'no',
 'nosso',
 'sistema',
 ':',
 'java',
 'Para',
 'salvar',
 'um',
 'novo',
 'usuário',
 ',',
 'várias',
 'validações',
 'são',
 'feitas',
 ',',
 'como',
 'por',
 'exemplo',
 ':',
 'Ver',
 'se',
 'o',
 'nome',
 'só',
 'contém',
 'letras',
 ',',
 '[',
 '*',
 '*',
 'o',
 'CPF',
 'só',
 'números',
 '*',
 '*',
 ']',
 'e',
 'ver',
 'se']

### 04. Separando palavras de tokens

In [5]:
from typing import List

def separa_palavras(lista_tokens: List[str]) -> List[str]:
    lista_palavras = list()
    for token in lista_tokens:
        if token.isalpha():
            lista_palavras.append(token)
    return lista_palavras

### 05. Contando palavras do Corpus

In [8]:
lista_palavras = separa_palavras(lista_tokens)
len(lista_palavras)

403104

### 06. Normalização

In [9]:
def normalizacao(lista_palavras: List[str]) -> List[str]:
    lista_normalizada = list()
    for palavra in lista_palavras:
        lista_normalizada.append(palavra.lower())
    return lista_normalizada

In [10]:
lista_normalizada = normalizacao(lista_palavras)
lista_normalizada[:5]

['imagem', 'temos', 'a', 'seguinte', 'classe']

### 07. Tipos de palavras

In [11]:
print(len(set(lista_normalizada)))

18465


## 03. Desenvolvendo e testando o corretor

### 03. Fatiando strings

In [34]:
palavra_exemplo = "lgica"

def gerador_palavras(palavra: str) -> List[str]:
    fatias = list()
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i], palavra[i:]))
    print(fatias)

In [35]:
gerador_palavras(palavra_exemplo)

[('', 'lgica'), ('l', 'gica'), ('lg', 'ica'), ('lgi', 'ca'), ('lgic', 'a'), ('lgica', '')]


### 04. Operação de inserção

In [36]:
from typing import Tuple

def insere_letras(fatias: List[Tuple]) -> List:
    novas_palavras = list()
    letras = "abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôòõúûùũç"
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D)
    return novas_palavras

In [37]:
def gerador_palavras(palavra: str) -> List[str]:
    def insere_letras(fatias: List[Tuple]) -> List:
        novas_palavras = list()
        letras = "abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôòõúûùũç"
        for E, D in fatias:
            for letra in letras:
                novas_palavras.append(E + letra + D)
        return novas_palavras

    fatias = list()
    for i in range(len(palavra) + 1):
        fatias.append((palavra[:i], palavra[i:]))
    palavras_geradas = insere_letras(fatias)
    return palavras_geradas

In [38]:
gerador_palavras(palavra_exemplo)[:10]

['algica',
 'blgica',
 'clgica',
 'dlgica',
 'elgica',
 'flgica',
 'glgica',
 'hlgica',
 'ilgica',
 'jlgica']

### 06. Construindo a função corretor

In [17]:
def corretor(palavra: str) -> str:
    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

### 07. Probabilidade das palavras geradas

In [18]:
frequencia = nltk.FreqDist(lista_normalizada)
total_palavras = len(lista_normalizada)

In [19]:
def probabilidade(palavra_gerada: str):
    return frequencia[palavra_gerada]/total_palavras

In [39]:
def corretor(palavra: str) -> str:
    def probabilidade(palavra_gerada: str):
        return frequencia[palavra_gerada]/total_palavras

    palavras_geradas = gerador_palavras(palavra)
    palavra_correta = max(palavras_geradas, key=probabilidade)
    return palavra_correta

In [40]:
corretor(palavra_exemplo)

'lógica'

## 04. Avaliando a qualidade do corretor

### 02. Preparando dados de teste

In [49]:
def cria_dados_teste(nome_arquivo: str) -> List[str]:
    lista_palavras_teste = list()
    f = open(nome_arquivo, "r")
    for linha in f:
        correta, errada = linha.split(" ")
        lista_palavras_teste.append((correta, errada))
    f.close()
    return lista_palavras_teste

In [51]:
lista_teste = cria_dados_teste("./dados/palavras.txt")
lista_teste[:10]

[('podemos', 'pyodemos\n'),
 ('esse', 'esje\n'),
 ('já', 'jrá\n'),
 ('nosso', 'nossov\n'),
 ('são', 'sãêo\n'),
 ('dos', 'dosa\n'),
 ('muito', 'muifo\n'),
 ('imagem', 'iômagem\n'),
 ('sua', 'ósua\n'),
 ('também', 'tambéùm\n')]

### 03. Avaliando o corretor

In [54]:
def avaliador(testes: List[str]) -> None:
    numero_palavras = len(testes)
    acertou = 0
    for correta, errada in testes:
        palavra_corrigida = corretor(errada)
        if palavra_corrigida == correta:
            acertou += 1
    taxa_acerto = round(acertou*100/numero_palavras, 2)
    print(f"{taxa_acerto}% de {numero_palavras} palavras")

In [55]:
avaliador(lista_teste)

0.0% de 186 palavras
